In [1]:
from dotenv import load_dotenv
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

c:\Users\91885\anaconda3\envs\intern\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
slides = [
    "Meena met Mathur in Lodhi Garden, under the rain.",
    "They shared chai and laughter in Khan Market.",
    "Mathur reveals he’s leaving for London in a week."
]

# Convert to LangChain Documents
documents = [Document(page_content=slide) for slide in slides]

In [3]:
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [4]:
print(PINECONE_API_KEY)

pcsk_8EzX9_4dxK8xMuCVmFi1FwuNPxRAMwWF7HAwfhiA3VYNbGARMDPvY8CkqCqGSUVqUPKbG


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
def download_huggingface_embeddding():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-mpnet-base-v2")
    return embeddings

In [7]:
embedding = download_huggingface_embeddding()

C:\Users\91885\AppData\Local\Temp\ipykernel_7384\3429849932.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [8]:
import os

In [9]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "hii"

pc.create_index(
    name=index_name,
    dimension=768,  # Replace with your model dimensions
    metric="cosine",  # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "hii",
    "metric": "cosine",
    "host": "hii-hu1niv7.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 768,
    "deletion_protection": "disabled",
    "tags": null
}

In [10]:


# Create the vector store
docsearch = PineconeVectorStore.from_documents(
    documents=documents,
    index_name=index_name,
    embedding=embedding
)

In [11]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name="hii",
    embedding=embedding

)

In [12]:
retriever = docsearch.as_retriever(
    search_type="similarity", search_kwargs={"k": 1})

In [15]:
user_prompt = "Continue the story with emotions"
retrieved_doc = retriever.invoke(user_prompt)

In [16]:
retrieved_doc

[Document(id='d52a4b44-a2f3-4d88-8eaf-0086592f30da', metadata={}, page_content='They shared chai and laughter in Khan Market.')]

In [20]:
user_instruction = "Continue the story with more emotional tension and vulnerability." # [cite: 1]
print(f"User Instruction: {user_instruction}\n")

User Instruction: Continue the story with more emotional tension and vulnerability.



In [21]:
retrieved_docs = retriever.invoke(user_instruction)

In [22]:
retrieved_docs

[Document(id='ea403274-f59f-4200-9b2d-27e86cdf33b5', metadata={}, page_content='Mathur reveals he’s leaving for London in a week.')]

In [19]:
# Assume 'retriever' is already defined and configured as in your notebook
# Assume 'embedding' is also defined

# --- Previous parts of your script ---
# (Loading slides, creating documents, setting up Pinecone, embedding, retriever)

# Use the user instruction from the PDF for consistent testing
user_instruction = "Continue the story with more emotional tension and vulnerability." # [cite: 1]
print(f"User Instruction: {user_instruction}\n")

# 1. Retrieve the single most relevant slide
# Ensure your retriever is correctly configured to return k=1
# If you are still getting two documents, you need to debug that part of your
# PineconeVectorStore setup or how documents were indexed.
# For this example, we'll assume `retrieved_docs` contains ONE document.
retrieved_docs = retriever.invoke(user_instruction)

# Check if documents were retrieved and handle potential issues
if not retrieved_docs:
    print("No relevant slide found.")
else:
    # Assuming the retriever correctly returns only one document as per k=1
    # If it's returning a list, take the first element.
    # You might need to add error handling if it's not a list or is empty.
    most_relevant_slide_doc = retrieved_docs[0] if isinstance(retrieved_docs, list) else retrieved_docs
    relevant_slide_content = most_relevant_slide_doc.page_content

    # 2. Print the "Most Relevant Slide"
    # Find which of the original slides matches the retrieved content to get its "Slide X" label.
    # The starter slides are:
    # Slide 1: Meena met Mathur in Lodhi Garden, under the rain.
    # Slide 2: They shared chai and laughter in Khan Market.
    # Slide 3: Mathur reveals he's leaving for London in a week. [cite: 1]

    slide_label = ""
    if relevant_slide_content == "Meena met Mathur in Lodhi Garden, under the rain.":
        slide_label = "Slide 1: "
    elif relevant_slide_content == "They shared chai and laughter in Khan Market.":
        slide_label = "Slide 2: "
    elif relevant_slide_content == "Mathur reveals he’s leaving for London in a week.": # Content from your slides variable
        slide_label = "Slide 3: "

    print("Most Relevant Slide:") # [cite: 1]
    print(f'{slide_label}"{relevant_slide_content}"\n') # [cite: 1]

    # 3. Print the "Final Prompt"
    final_prompt = (
        "You are continuing a love story between Meena and Mathur.\n" # [cite: 1]
        f"Previously, {relevant_slide_content.rstrip('.')}.\n" # [cite: 1] Modified to use retrieved content
        f"Now, write the next slide with more emotional tension and vulnerability." # [cite: 1] (using the PDF's instruction)
    )
    print("Final Prompt:") # [cite: 1]
    print(final_prompt)

# --- For dynamic input as per PDF objective 3 ---
# To "Accepts a new user instruction" dynamically, you could use:
# user_instruction_dynamic = input("Enter your instruction to continue the story: ")
# Then use user_instruction_dynamic in the retriever.invoke() call

User Instruction: Continue the story with more emotional tension and vulnerability.

Most Relevant Slide:
Slide 3: "Mathur reveals he’s leaving for London in a week."

Final Prompt:
You are continuing a love story between Meena and Mathur.
Previously, Mathur reveals he’s leaving for London in a week.
Now, write the next slide with more emotional tension and vulnerability.
